In [10]:
import analyzer
from sklearn.feature_extraction.text import CountVectorizer
import io
import tokenize

In [ ]:
# %load analyzer.py
import io
import tokenize
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_params = {
    "ngram_range": (1,3)
}

removed_itens = ['NEWLINE', 'ENDMARKER', "NL", 'COMMENT', 'ERRORTOKEN']
allowed_itens = ['NAME', 'OP', 'INDENT', 'DEDENT', 'STRING', 'NUMBER']

def python_analyzer(doc):
    words = []
    not_found = []
    #print(vectorizer_params)
    vectorizer = CountVectorizer(**vectorizer_params)
    file = io.StringIO(doc)
    try:
        for token in tokenize.generate_tokens(file.readline):
            token_type = tokenize.tok_name[token[0]]

            # Redundant conditional to make sure we're getting all the token types
            if token_type not in removed_itens:
                if token_type in allowed_itens:
                    # If it's a variable or reserved name, keep it
                    if token_type == "NAME":
                        words.append(token[1])
                    elif token_type == "INDENT":
                        # Adding indent for all indentations
                        words.append("is_indent")
                    elif token_type == "DEDENT":
                        # Adding dedent for all indentations
                        words.append("is_dedent")
                    elif token_type == "STRING":
                        # Adding is_string for every string
                        words.append("is_string")
                    elif token_type == "NUMBER":
                        # Adding is_number for every number:
                        words.append("is_number")
                    elif token_type == "OP":
                        # If it's operator, then we'll divide in several types
                        lookup = {
                            "+": "is_op_arit",
                            "+=": "is_op_arit",
                            "-": "is_op_arit",
                            "*": "is_op_arit",
                            "**": "is_op_arit",
                            "/": "is_op_arit",
                            "//": "is_op_arit",
                            "%": "is_op_arit",
                            ">": "is_op_logic",
                            "<": "is_op_logic",
                            ">=": "is_op_logic",
                            "<=": "is_op_logic",
                            "==": "is_op_logic",
                            "-=": "is_op_logic",
                            "!=": "is_op_logic",
                            "[": "is_list",
    #                         "]": "is_list",
                            "{": "is_dict",
    #                         "}": "is_dict",
                            ".": "is_class",
                            "=": "is_attribution",
                            ":": "is_block"
                        }
                        try:
                            words.append(lookup[token[1]])
                        except KeyError:
                            not_found.append(token[1])
    #     print("not found: %s" % set(not_found))
    except (IndentationError, tokenize.TokenError):
        pass
    return vectorizer._word_ngrams(words)


In [2]:
text = """def light(A, B):
    if A == 1 and B == 1:
        return True
    else:
        return False"""

In [6]:
v = CountVectorizer(analyzer = analyzer.python_analyzer)

In [8]:
v.analyzer(text)

['def',
 'light',
 'A',
 'B',
 'is_block',
 'is_indent',
 'if',
 'A',
 'is_op_logic',
 'is_number',
 'and',
 'B',
 'is_op_logic',
 'is_number',
 'is_block',
 'is_indent',
 'return',
 'True',
 'is_dedent',
 'else',
 'is_block',
 'is_indent',
 'return',
 'False',
 'is_dedent',
 'is_dedent',
 'def light',
 'light A',
 'A B',
 'B is_block',
 'is_block is_indent',
 'is_indent if',
 'if A',
 'A is_op_logic',
 'is_op_logic is_number',
 'is_number and',
 'and B',
 'B is_op_logic',
 'is_op_logic is_number',
 'is_number is_block',
 'is_block is_indent',
 'is_indent return',
 'return True',
 'True is_dedent',
 'is_dedent else',
 'else is_block',
 'is_block is_indent',
 'is_indent return',
 'return False',
 'False is_dedent',
 'is_dedent is_dedent',
 'def light A',
 'light A B',
 'A B is_block',
 'B is_block is_indent',
 'is_block is_indent if',
 'is_indent if A',
 'if A is_op_logic',
 'A is_op_logic is_number',
 'is_op_logic is_number and',
 'is_number and B',
 'and B is_op_logic',
 'B is_op_logi

In [12]:
file = io.StringIO(text)
for t in tokenize.generate_tokens(file.readline):
    print(t)

TokenInfo(type=1 (NAME), string='def', start=(1, 0), end=(1, 3), line='def light(A, B):\n')
TokenInfo(type=1 (NAME), string='light', start=(1, 4), end=(1, 9), line='def light(A, B):\n')
TokenInfo(type=53 (OP), string='(', start=(1, 9), end=(1, 10), line='def light(A, B):\n')
TokenInfo(type=1 (NAME), string='A', start=(1, 10), end=(1, 11), line='def light(A, B):\n')
TokenInfo(type=53 (OP), string=',', start=(1, 11), end=(1, 12), line='def light(A, B):\n')
TokenInfo(type=1 (NAME), string='B', start=(1, 13), end=(1, 14), line='def light(A, B):\n')
TokenInfo(type=53 (OP), string=')', start=(1, 14), end=(1, 15), line='def light(A, B):\n')
TokenInfo(type=53 (OP), string=':', start=(1, 15), end=(1, 16), line='def light(A, B):\n')
TokenInfo(type=4 (NEWLINE), string='\n', start=(1, 16), end=(1, 17), line='def light(A, B):\n')
TokenInfo(type=5 (INDENT), string='    ', start=(2, 0), end=(2, 4), line='    if A == 1 and B == 1:\n')
TokenInfo(type=1 (NAME), string='if', start=(2, 4), end=(2, 6), lin